In [1]:
%pip install atproto

   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 27.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
from atproto import Client, client_utils

In [2]:
client = Client()
profile = client.login('mehoha6465.bsky.social', 'idontcareaboutthisaccount')

In [2]:
import pandas as pd

df = pd.read_csv('initialTeamData.csv')

# Convert to a dictionary where each team is a key and players are a list of values
team_dict = df.groupby('team')['player'].apply(list).to_dict()

team_dict

{'Betboom': ['Zorte', 'Magnojez', 'Nafay', 'S1ren', 'KaiRON-'],
 'Cloud9': ['Ax1Le', 'Boombl4', 'HeavyGod', 'ICY', 'interz'],
 'ECLOT': ['Dytor', 'nbqq', 'FORSYY', 'Blytz', 'kreaz'],
 'FaZe Clan': ['rain', 'broky', 'karrigan', 'ropz', 'frozen'],
 'Fnatic': ['KRIMZ', 'bodyy', 'matys', 'blameF', 'nawwk'],
 'GamerLegion': ['Fl4mus', 'Zorte', 'Tauson', 'Sl3nd', 'Volt'],
 'Natus Vincere': ['b1t', 'Aleksib', 'jL', 'iM', 'w0nderful'],
 'Nemiga': ['Xant3r', '1eer', 'Riskyb0b', 'khaN', 'zweih'],
 'Rebels': ['Innocent', 'Casey', 'Olimp', 'Flayy', 'Kisserek'],
 'SAW': ['MUTiRiS', 'rmn', 'ewjerkz', 'story', 'Ag1l']}

In [21]:
tags = ['Perfect World Shanghai Major 2024: European RMR A', 'Perfect World Shanghai Major 2024']

In [ ]:
from datetime import datetime
from atproto_client import models
import csv

csv_data = []

# Iterate through tags one by one
# for tag in tags:
# print(f"Querying for tag: {tag}")
for team, players in team_dict.items():
    print(f"Querying for team: {team} with tag: {tags}")
    team_posts = client.app.bsky.feed.search_posts(
        params=models.AppBskyFeedSearchPosts.Params(
            q=team,
            since=datetime.strptime(f"2024-11-15", "%Y-%m-%d").isoformat() + ".000000Z",
            until=datetime.strptime(f"2024-11-20", "%Y-%m-%d").isoformat().replace("T00:00:00", '') + "T23:59:59.999999Z",
            tag=tags 
        )
    )
    for post in team_posts.posts:
        # Add team posts to CSV data
        csv_data.append({
            "team": team,
            "post": post.record.text,
            "post_author": post.author.display_name,
            "post_time": post.record.created_at,
            "tag": tags
        })
    for player in players:
        # Collect posts for each player
        print(f"Querying for player: {player} in team {team} with tag: {tag}")
        player_posts = client.app.bsky.feed.search_posts(
            params=models.AppBskyFeedSearchPosts.Params(
                q=player,
                since=datetime.strptime(f"2024-11-15", "%Y-%m-%d").isoformat() + ".000000Z",
                until=datetime.strptime(f"2024-11-20", "%Y-%m-%d").isoformat().replace("T00:00:00", '') + "T23:59:59.999999Z",
                tag=tags 
            )
        )
        for post in player_posts.posts:
            # Add player posts to the team's posts
            csv_data.append({
                "team": team,
                "post": post.record.text,
                "post_author": post.author.display_name,
                "post_time": post.record.created_at,
                "tag": tags
            })

    # Write the CSV
    output_file = f'tagged_posts_cs2.csv'
    with open(output_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=["team", "post", "post_author", "post_time", "tag"])
        writer.writeheader()
        writer.writerows(csv_data)

    print(f"Tagged posts saved to {output_file}")

Querying for team: Betboom with tag: ['Perfect World Shanghai Major 2024: European RMR A', 'Perfect World Shanghai Major 2024']
Querying for player: Zorte in team Betboom with tag: European RMR A
Querying for player: Magnojez in team Betboom with tag: European RMR A
Querying for player: Nafay in team Betboom with tag: European RMR A
Querying for player: S1ren in team Betboom with tag: European RMR A
Querying for player: KaiRON- in team Betboom with tag: European RMR A
Tagged posts saved to tagged_posts_cs2.csv
Querying for team: Cloud9 with tag: ['Perfect World Shanghai Major 2024: European RMR A', 'Perfect World Shanghai Major 2024']
Querying for player: Ax1Le in team Cloud9 with tag: European RMR A
Querying for player: Boombl4 in team Cloud9 with tag: European RMR A
Querying for player: HeavyGod in team Cloud9 with tag: European RMR A
Querying for player: ICY in team Cloud9 with tag: European RMR A
Querying for player: interz in team Cloud9 with tag: European RMR A
Tagged posts saved

In [ ]:
import pandas as pd
from collections import defaultdict

csposts = pd.read_csv('tagged_posts_cs2.csv')

In [24]:
print(csposts['team'].value_counts(), '\n')

team
Rebels           84
SAW              76
Natus Vincere    64
GamerLegion      63
FaZe Clan        62
Cloud9           51
Nemiga           41
Fnatic           38
Betboom          36
ECLOT            20
Name: count, dtype: int64 



In [29]:
csposts['post_time'] = pd.to_datetime(csposts['post_time'], format='mixed')
cspostsSorted = csposts.sort_values(by='post_time')

BBVGLCutoff = pd.to_datetime('2024-11-20T08:25:00').tz_localize('UTC') 

BBVGLDF = csposts[csposts['team'].isin(['Betboom', 'GamerLegion'])]
BBVGLDF = BBVGLDF[BBVGLDF['post_time'] < BBVGLCutoff]

FnVECLCutoff = pd.to_datetime('2024-11-20T00:00:00').tz_localize('UTC') 

FnVECLDF = csposts[csposts['team'].isin(['Fnatic', 'ECLOT'])]
FnVECLDF = FnVECLDF[FnVECLDF['post_time'] < FnVECLCutoff]

FzVC9Cutoff = pd.to_datetime('2024-11-19T07:00:00').tz_localize('UTC') 

FzVC9DF = csposts[csposts['team'].isin(['FaZe Clan', 'Cloud9'])]
FzVC9DF = FzVC9DF[FzVC9DF['post_time'] < FzVC9Cutoff]

NVVSAWCutoff = pd.to_datetime('2024-11-19T00:20:00').tz_localize('UTC') 

NVVSAWDF = csposts[csposts['team'].isin(['Natus Vincere', 'SAW'])]
NVVSAWDF = NVVSAWDF[NVVSAWDF['post_time'] < NVVSAWCutoff]

NGVRebCutoff = pd.to_datetime('2024-11-18T01:05:00').tz_localize('UTC') 

NGVRebDF = csposts[csposts['team'].isin(['Nemiga', 'Rebels'])]
NGVRebDF = NGVRebDF[NGVRebDF['post_time'] < NGVRebCutoff]



BBVGLDF['source'] = 'BlueSky'
FnVECLDF['source'] = 'BlueSky'
FzVC9DF['source'] = 'BlueSky'
NVVSAWDF['source'] = 'BlueSky'
NGVRebDF['source'] = 'BlueSky'

combined_cs_df = pd.concat([BBVGLDF, FnVECLDF, FzVC9DF, NVVSAWDF, NGVRebDF])

# combined_cs_df = combined_cs_df.sort_values(by='post_time')

combined_cs_df.reset_index(drop=True, inplace=True)

combined_cs_df.to_csv('cs_timecutoff_posts.csv', index=False)

print(combined_cs_df["team"].value_counts())


team
Betboom          30
GamerLegion      27
Fnatic           22
ECLOT            13
Cloud9           12
SAW              12
Natus Vincere     8
FaZe Clan         7
Rebels            6
Nemiga            5
Name: count, dtype: int64


In [34]:
csposts = pd.read_csv('BlueSky_cs_timecutoff_posts.csv')

csposts.dropna(inplace=True)

annotation_map = {1: 'positive', 2: 'negative', 3: 'neutral'}

csposts['Annotations'] = pd.to_numeric(csposts['Annotations'], errors='coerce')

# Group by team and calculate required metrics
simplified_data = csposts.groupby('team').agg(
    post_count=('post', 'count'),
    positive_posts=('Annotations', lambda x: (x == 1).sum()),
    negative_posts=('Annotations', lambda x: (x == 2).sum()),
    neutral_posts=('Annotations', lambda x: (x == 3).sum())
).reset_index()

rebels_row = {'team': 'Rebels', 'post_count': 0, 'positive_posts': 0, 'negative_posts': 0, 'neutral_posts': 0}

simplified_data.loc[len(simplified_data)] = rebels_row


performance_dict = {'Betboom': 0, 'GamerLegion': 1, 'Fnatic': 1, 'ECLOT': 0, 'Cloud9': 0, 'SAW': 0, 'Natus Vincere': 1, 'FaZe Clan': 1, 'Rebels': 0, 'Nemiga': 1}

simplified_data['performance'] = simplified_data['team'].map(performance_dict)

simplified_data.to_csv("team_data.csv")

simplified_data

,team,post_count,positive_posts,negative_posts,neutral_posts,performance
0,Betboom,19,5,3,11,0
1,Cloud9,8,4,0,4,0
2,ECLOT,12,3,2,7,0
3,FaZe Clan,7,4,1,2,1
4,Fnatic,12,4,4,4,1
5,GamerLegion,20,7,4,8,1
6,Natus Vincere,5,4,0,1,1
7,Nemiga,5,1,3,1,1
8,SAW,1,1,0,0,0
9,Rebels,0,0,0,0,0


In [4]:
!pip install pandas
!pip install matplotlib
!pip install plotly
!pip install selenium
!pip install bs4
!pip install praw
!pip install textblob
!pip install nltk
!pip install networkx
!pip install scipy

In [1]:
"""This cell imports necessary Python modules and performs initial configuration
"""

# Data manipulation libraries
import json
import pandas as pd 
import csv


# Visualization and Interaction
# import matplotlib.pyplot as plt
# plt.style.use('ggplot')
from IPython.display import set_matplotlib_formats, display, clear_output, HTML
set_matplotlib_formats('retina')
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot 
init_notebook_mode(connected=True)
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import VBox, HBox, Button, HTML, Label


# Computation libraries 
import numpy as np
import re
import random


# Graph analysis
# import networkx as nx
# import community


# System related
# import sys
# import warnings;
# warnings.filterwarnings('ignore')
import io
import platform
from pathlib import Path
import os
from getpass import getpass
# from joblib import Parallel, delayed


# Datetime libraries
from datetime import datetime
import time
from pytz import timezone


# NLP dependencies
# import spacy
# from spacy.tokenizer import Tokenizer
# nlp = spacy.load('en')
# tokenizer = Tokenizer(nlp.vocab)
# from langdetect import detect


# Scraping libraries
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup


# Machine learning libraries
# from sklearn import datasets
# from sklearn import linear_model
# from sklearn.feature_selection import f_regression, mutual_info_regression
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report


# Logging
import logging 
logging.basicConfig(level=logging.INFO)


# For Reddit
import praw
from praw.models import MoreComments

C:\Users\dunca\AppData\Local\Temp\ipykernel_6620\4234844595.py:14: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('retina')


## Parameters

In [18]:
"""This cell defines global variables and parameters used throughout the playbook
"""

# Set this to True if you want to watch Selenium scrape pages
# WATCH_SCRAPING = True

# Set this to True if you want to use incognito mode
# USE_INCOGNITO = True

# Number of posts 
post_limit = 1000

# The data is written 
RAW_DATA_DIRECTORY = Path("")

# Setup logging level
LOGGING_LEVEL = logging.INFO 
logging.basicConfig(level=LOGGING_LEVEL)

## Functions and Classes

In [17]:
"""This cell defines functions and classes used throughout the playbook
"""

def __init__(self, client_id, client_secret, user_agent, password):
    self.client_id = client_id
    self.client_secret = client_secret
    self.user_agent = user_agent
    self.password = password


def token(client_id, client_secret, user_agent):
    reddit = praw.Reddit(client_id=client_id,
                         client_secret=client_secret,
                         user_agent=user_agent)
    if (reddit != False):
        print("Successful token")
    else:
        print("Failed token")
    return reddit

def search_subreddit(reddit, search_term, sort_type, time_limit, post_limit, sub_reddit):
    """
    GRAB REDDIT POSTS BY SEARCH TERM
    search_term = any boolean search #https://www.reddit.com/dev/api/
    sort_type = 'relevance', 'hot', 'top', 'new', 'comments'
    time_limit = 'all', 'hour', 'day', 'week', 'month', 'year'
    post_limit = 1000 maximum
    """

    posts = []
    subreddit = reddit.subreddit(sub_reddit)
    for post in subreddit.search(search_term, sort=sort_type, time_filter=time_limit, limit=post_limit):
        posts.append([post.subreddit, post.id, post.title, post.selftext, post.author, post.url, post.permalink,
                      post.num_comments, post.created, post.score, post.distinguished, post.is_original_content,
                      post.upvote_ratio, post.link_flair_text])
    posts = pd.DataFrame(posts,
                         columns=['subreddit', 'post_id', 'title', 'post_body', 'post_author', 'url', 'post_permalink',
                                  'num_comments', 'post_created', 'post_score', 'post_distinguished',
                                  'original_content', 'upvote_ratio', 'flair_text'])
    posts['post_created'] = pd.to_datetime(posts['post_created'], unit='s')
    posts['scrape_time'] = datetime.now()
    posts[['subreddit', 'post_id', 'title', 'post_author',
           'post_body', 'url', 'post_permalink', 'flair_text']] = posts[['subreddit', 'post_id', 'title', 'post_author',
                                                                         'post_body', 'url', 'post_permalink',
                                                                         'flair_text']].astype(str)
    return posts




## System-dependent Configuration

In [3]:
"""This cell defines system-dependent configuration such as those different in Linux vs. Windows
"""

# Get the system information from the OS
PLATFORM_SYSTEM = platform.system()

# Darwin is macOS
if PLATFORM_SYSTEM == "Darwin":
    EXECUTABLE_PATH = Path("../dependencies/chromedriver")
elif PLATFORM_SYSTEM == "Windows":
    EXECUTABLE_PATH = Path("../dependencies/chromedriver.exe")
else:
    logging.critical("Chromedriver not found or Chromedriver is outdated...")
    exit()
    
tz = timezone('US/Eastern')    

# Collect Data

## Collect Reddit Submissions and Comments

In [28]:
"""This cell retrieves page posts and comments, for a given page.
"""

def main():
    # Credentials (create client_id, client_secret, user_agent by following https://praw.readthedocs.io/en/latest/getting_started/quick_start.html)
    client_id = 'Mo7nM-yhqz1cww'
    client_secret = '-4O8dN-Emr5PdkQfy8ifCSVlihE'
    user_agent = 'collectPosts'

    ''' Designate input parameters for functions
    search terms = key terms to search ALL of reddit
    subreddits = subreddits to collect
    search_sort_type = 'relevance', 'hot', 'top', 'new', 'comments'
    sub_sort_type = 'hot', 'top', 'new', 'gilded', 'rising', 'controversial'
    time_limit = 'all', 'hour', 'day', 'week', 'month', 'year'
    post_limit = 10
    '''

    sub_sort_type = 'top'  # , 'top', 'new', 'gilded', 'rising', 'controversial'
    search_sort_type = 'top'  # , 'hot', 'top', 'new', 'comments', 'relevance'
    time_limit = 'month'  # , 'hour', 'day', 'week', 'month', 'year'
    
    search_terms = ['Zorte', 'Magnojez', 'Nafay', 'S1ren', 'KaiRON-', 'Ax1Le', 'Boombl4', 'HeavyGod', 'ICY', 'interz', 'Dytor', 'nbqq', 'FORSYY', 'Blytz', 'kreaz', 'rain', 'broky', 'karrigan', 'ropz', 'frozen', 'KRIMZ', 'bodyy', 'matys', 'blameF', 'nawwk', 'Fl4mus', 'Zorte', 'Tauson', 'Sl3nd', 'Volt', 'b1t', 'Aleksib', 'jL', 'iM', 'w0nderful', 'Xant3r', '1eer', 'Riskyb0b', 'khaN', 'zweih', 'Innocent', 'Casey', 'Olimp', 'Flayy', 'Kisserek','MUTiRiS', 'rmn', 'ewjerkz', 'story', 'Ag1l']
    subreddits = ['cs2', 'GlobalOffensive']

    # input_search_terms = input("Enter search terms (seperated by spaces): ")
    # search_terms = input_search_terms.split()
    
    # input_subreddits = input("Enter subreddits (separated by spaces): ")
    # subreddits = input_subreddits.split()
    
    ''' Collect posts & corresponding comments
    '''
    # Create client
    r = token(client_id, client_secret, user_agent)
    list_posts_df = []
    try:
        for query in search_terms:
            for subreddit in subreddits:
                print(subreddit)
                post_df = search_subreddit(r, query, search_sort_type, time_limit, post_limit, subreddit)
                print("Grabbed", len(post_df), "posts with search term:", query, "In subreddit: ", subreddit)
                list_posts_df.append(post_df)
    except:
        pass

    new_posts = pd.concat(list_posts_df)
    print("Number of posts: ", new_posts.shape[0])

    # File output for posts
    filename_csv = "cs_reddit_posts.csv"
    new_posts.to_csv(str(RAW_DATA_DIRECTORY / filename_csv), index=False)
    print("Exported posts to CSV")

if __name__ == "__main__":
    main()

Successful token
cs2
Grabbed 0 posts with search term: Zorte In subreddit:  cs2
GlobalOffensive
Grabbed 11 posts with search term: Zorte In subreddit:  GlobalOffensive
cs2
Grabbed 0 posts with search term: Magnojez In subreddit:  cs2
GlobalOffensive
Grabbed 12 posts with search term: Magnojez In subreddit:  GlobalOffensive
cs2
Grabbed 0 posts with search term: Nafay In subreddit:  cs2
GlobalOffensive
Grabbed 0 posts with search term: Nafay In subreddit:  GlobalOffensive
cs2
Grabbed 0 posts with search term: S1ren In subreddit:  cs2
GlobalOffensive
Grabbed 11 posts with search term: S1ren In subreddit:  GlobalOffensive
cs2
Grabbed 0 posts with search term: KaiRON- In subreddit:  cs2
GlobalOffensive
Grabbed 1 posts with search term: KaiRON- In subreddit:  GlobalOffensive
cs2
Grabbed 1 posts with search term: Ax1Le In subreddit:  cs2
GlobalOffensive
Grabbed 16 posts with search term: Ax1Le In subreddit:  GlobalOffensive
cs2
Grabbed 1 posts with search term: Boombl4 In subreddit:  cs2
Glob

C:\Users\dunca\AppData\Local\Temp\ipykernel_6620\713749511.py:47: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



In [80]:
def cut_time(df):
    csposts = df
    BBVGLCutoff = pd.Timestamp('2024-11-20T08:25:00')

    BBVGLDF = csposts[csposts['team'].isin(['Betboom', 'GamerLegion'])]
    BBVGLDF = BBVGLDF[BBVGLDF['post_created'] < BBVGLCutoff]

    FnVECLCutoff = pd.Timestamp('2024-11-20T00:00:00')

    FnVECLDF = csposts[csposts['team'].isin(['Fnatic', 'ECLOT'])]
    FnVECLDF = FnVECLDF[FnVECLDF['post_created'] < FnVECLCutoff]

    FzVC9Cutoff = pd.Timestamp('2024-11-19T07:00:00')

    FzVC9DF = csposts[csposts['team'].isin(['FaZe Clan', 'Cloud9'])]
    FzVC9DF = FzVC9DF[FzVC9DF['post_created'] < FzVC9Cutoff]

    NVVSAWCutoff = pd.Timestamp('2024-11-19T00:20:00')

    NVVSAWDF = csposts[csposts['team'].isin(['Natus Vincere', 'SAW'])]
    NVVSAWDF = NVVSAWDF[NVVSAWDF['post_created'] < NVVSAWCutoff]

    NGVRebCutoff = pd.Timestamp('2024-11-18T01:05:00')

    NGVRebDF = csposts[csposts['team'].isin(['Nemiga', 'Rebels'])]
    NGVRebDF = NGVRebDF[NGVRebDF['post_created'] < NGVRebCutoff]

    BBVGLDF['source'] = 'reddit'
    FnVECLDF['source'] = 'reddit'
    FzVC9DF['source'] = 'reddit'
    NVVSAWDF['source'] = 'reddit'
    NGVRebDF['source'] = 'reddit'

    combined_cs_df = pd.concat([BBVGLDF, FnVECLDF, FzVC9DF, NVVSAWDF, NGVRebDF])

    # combined_cs_df = combined_cs_df.sort_values(by='post_time')

    combined_cs_df.reset_index(drop=True, inplace=True)

    return combined_cs_df


In [85]:
df = pd.read_csv('cs_reddit_posts.csv')

df['post_created'] = pd.to_datetime(df['post_created'])


tourney_start = pd.Timestamp('2024-11-15') #Couple days before like above
tourney_end = pd.Timestamp('2024-11-20')

df = df[tourney_start <= df['post_created']]
df = df[df['post_created'] <= tourney_end]
df['source'] = 'Reddit'

teamlist = ['Natus Vincere', 'FaZe Clan', 'SAW', 'Fnatic', 'Cloud9', 'Rebels', 'Betboom', 'GamerLegion', 'ECLOT', 'Nemiga']

team_players = {
    'Betboom': ['Zorte', 'Magnojez', 'Nafay', 'S1ren', 'KaiRON-'],
    'Cloud9': ['Ax1Le', 'Boombl4', 'HeavyGod', 'ICY', 'interz'],
    'ECLOT': ['Dytor', 'nbqq', 'FORSYY', 'Blytz', 'kreaz'],
    'FaZe Clan': ['rain', 'broky', 'karrigan', 'ropz', 'frozen'],
    'Fnatic': ['KRIMZ', 'bodyy', 'matys', 'blameF', 'nawwk'],
    'GamerLegion': ['Fl4mus', 'Zorte', 'Tauson', 'Sl3nd', 'Volt'],
    'Natus Vincere': ['b1t', 'Aleksib', 'jL', 'iM', 'w0nderful'],
    'Nemiga': ['Xant3r', '1eer', 'Riskyb0b', 'khaN', 'zweih'],
    'Rebels': ['Innocent', 'Casey', 'Olimp', 'Flayy', 'Kisserek'],
    'SAW': ['MUTiRiS', 'rmn', 'ewjerkz', 'story', 'Ag1l']
}

# Compile a regex pattern for both team and player names
pattern = '|'.join(teamlist + [player for players in team_players.values() for player in players])

# Ensure 'post_body' and 'title' are combined to account for both
df['content'] = df['post_body'] + " " + df['title']

# Filter posts containing any team or player
df_filtered = df[df['content'].str.contains(pattern, case=False, na=False)]

# Function to categorize posts by team and players
def categorize_post_to_multiple_teams(row, team_players):
    teams_mentioned = []
    for team, players in team_players.items():
        # Check if the team is mentioned
        if team.lower() in row['content'].lower():
            teams_mentioned.append(team)
        
        # Check if any of the players is mentioned
        for player in players:
            if player.lower() in row['content'].lower() and team not in teams_mentioned:
                teams_mentioned.append(team)
    return teams_mentioned  # Return a list of teams

# Apply categorization to assign posts to all mentioned teams
df_filtered['team'] = df_filtered.apply(lambda row: categorize_post_to_multiple_teams(row, team_players), axis=1)

# Explode the DataFrame so each row corresponds to one team and one post
df_result = df_filtered.explode('team')

df_cut = cut_time(df_result)


# Rename columns and save the result
df_cut = df_cut[['team', 'content']].rename(columns={'teams': 'team', 'content': 'post'})
df_cut["source"] = "reddit"

df_cut.drop_duplicates(inplace=True)

# Save to CSV
df_cut.to_csv('cs_posts_timecutoff_reddit.csv', index=False)

print(df_cut["team"].value_counts())

team
Natus Vincere    60
FaZe Clan        11
GamerLegion      10
SAW               8
Fnatic            7
Cloud9            6
ECLOT             6
Betboom           5
Nemiga            3
Rebels            3
Name: count, dtype: int64


C:\Users\dunca\AppData\Local\Temp\ipykernel_6620\3898105190.py:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [96]:
df = pd.read_csv('cs_annotated_posts_reddit.csv')

# print(df['team'].value_counts())

df['sentiment'] = pd.to_numeric(df['sentiment'], errors='coerce')

df.dropna(inplace=True)

df = df.drop(df[df['sentiment'] == 4].index)

# print(df['team'].value_counts())

df.drop_duplicates(inplace=True)

# print(df['team'].value_counts())

simplified_data_reddit = df.groupby('team').agg(
    post_count=('post', 'count'),
    positive_posts=('sentiment', lambda x: (x == 1).sum()),
    negative_posts=('sentiment', lambda x: (x == 2).sum()),
    neutral_posts=('sentiment', lambda x: (x == 3).sum())
).reset_index()

performance_dict = {'Betboom': 0, 'GamerLegion': 0, 'Fnatic': 0, 'ECLOT': 0, 'Cloud9': 0, 'SAW': 0, 'Natus Vincere': 0, 'FaZe Clan':0, 'Rebels': 0, 'Nemiga': 0} #Set all to 0 so we dont add a these together
simplified_data_reddit['performance'] = simplified_data_reddit['team'].map(performance_dict)

simplified_data_reddit

df_bluesky_simple = pd.read_csv('team_data.csv')

df_bluesky_simple

frames = [simplified_data_reddit, df_bluesky_simple]

result_df = pd.concat(frames).groupby('team', as_index=False).sum()

#Forgot I was going over only singular games not overall performance, left incase I revisit
# overall_performance_dict = {'Betboom': 8, 'GamerLegion': 7, 'Fnatic': 6, 'ECLOT': 11, 'Cloud9': 5, 'SAW': 10, 'Natus Vincere': 4, 'FaZe Clan':3, 'Rebels': 16, 'Nemiga': 12} 
# result_df['overall_performance'] = result_df['team'].map(overall_performance_dict)

now = pd.Timestamp.now()

team_creation = {'Betboom': (now - pd.Timestamp('2023-07-31')).days, 'GamerLegion': (now - pd.Timestamp('2019-03-29')).days, 'Fnatic': (now - pd.Timestamp('2004-07-23')).days,\
                 'ECLOT': (now - pd.Timestamp('2016-05-05')).days, 'Cloud9': (now - pd.Timestamp('2014-08-01')).days, 'SAW': (now - pd.Timestamp('2020-01-01')).days, \
                 'Natus Vincere': (now - pd.Timestamp('2009-12-17')).days, 'FaZe Clan':(now - pd.Timestamp('2016-01-20')).days, 'Rebels': (now - pd.Timestamp('2024-01-12')).days, \
                 'Nemiga': (now - pd.Timestamp('2017-06-05')).days} 
result_df['time_since_creation_days'] = result_df['team'].map(team_creation)


result_df.to_csv('team_performances_by_annotation.csv')

result_df

,team,post_count,positive_posts,negative_posts,neutral_posts,performance,time_since_creation_days
0,Betboom,24,5,3,16,0,490
1,Cloud9,14,4,0,10,0,3776
2,ECLOT,18,4,2,12,0,3133
3,FaZe Clan,11,4,1,6,1,3239
4,Fnatic,19,5,5,9,1,7437
5,GamerLegion,30,7,5,17,1,2075
6,Natus Vincere,21,5,1,15,1,5464
7,Nemiga,8,1,3,4,1,2737
8,Rebels,3,0,0,3,0,325
9,SAW,6,1,0,5,0,1797
